### Test BDD

In [1]:
import os
import psycopg2
import pandas as pd
import numpy
import sqlalchemy
from sqlalchemy import create_engine 

#### Création BDD

In [2]:
mot_passe = os.environ.get('pg_psw')


In [3]:
conn = psycopg2.connect(
   database="postgres", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

sql = '''CREATE database ProjetSPQR''';

cursor.execute(sql)
print("Base de données créée avec succès !")

conn.close()


Base de données créée avec succès !


In [4]:
conn = psycopg2.connect(
   database="projetspqr", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

#### Création des tables

In [5]:
def creer_table(conn, sql_creation_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creation_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")

In [6]:
sql_creer_table_personne = """
    CREATE TABLE IF NOT EXISTS personne (
    id text,
    nom_fr text,
    praenomen text,
    nomen text,
    cognomen text,
    gens text,
    sexe text,
    date_naissance text,
    lieu_naissance text,
    date_mort text,
    lieu_mort text,
    id_pere text,
    id_mere text
    );
"""

In [7]:
sql_creer_table_activite = """
    CREATE TABLE IF NOT EXISTS activite (
    personne_id text,
    activite text
    );
"""

In [8]:
sql_creer_table_poste = """
    CREATE TABLE IF NOT EXISTS poste (
    personne_id text,
    poste text
    );
"""

In [9]:
sql_creer_table_mariage = """
    CREATE TABLE IF NOT EXISTS mariage (
    id text,
    id_mari text,
    id_epouse text
    );
"""

In [10]:
sql_creer_table_oeuvre = """
    CREATE TABLE IF NOT EXISTS oeuvre (
    id text,
    titre_fr text,
    titre_lat text,
    auteur_id text,
    genre text
);
"""

In [11]:
creer_table(conn, sql_creer_table_personne)
creer_table(conn, sql_creer_table_activite)
creer_table(conn, sql_creer_table_poste)
creer_table(conn, sql_creer_table_mariage)
creer_table(conn, sql_creer_table_oeuvre)

La table a été crée avec succès
La table a été crée avec succès
La table a été crée avec succès
La table a été crée avec succès
La table a été crée avec succès


#### Dataframes

In [12]:
df_personne = pd.read_table("query_personne.csv", delimiter =",") 
print(df_personne.head()) 

                                     item                   itemLabel  \
0  http://www.wikidata.org/entity/Q727886  Marcus Valerius Maximianus   
1  http://www.wikidata.org/entity/Q728245   Saint Marcian of Syracuse   
2  http://www.wikidata.org/entity/Q728335      Lucius Caninius Gallus   
3  http://www.wikidata.org/entity/Q728349                   Severinus   
4  http://www.wikidata.org/entity/Q728527         Gaius Vibius Marsus   

  praenomenLabel nomenLabel cognomenLabel gensLabel sexeLabel  \
0         Marcus   Valerius           NaN   Valerii  masculin   
1            NaN        NaN           NaN       NaN  masculin   
2         Lucius   Caninius           NaN   Caninii  masculin   
3            NaN    Flavius           NaN    Flavii  masculin   
4          Caius     Vibius        Marsus     Vibii  masculin   

          date_naissance                          lieu_naissance  \
0   0200-01-01T00:00:00Z   http://www.wikidata.org/entity/Q15906   
1   0050-01-01T00:00:00Z  http://w

In [13]:
df_personne.rename(columns={"item": "id",
                            "itemLabel": "nom_fr",
                            "praenomenLabel": "praenomen",
                            "nomenLabel": "nomen",
                            "cognomenLabel": "cognomen",
                            "gensLabel": "gens",
                            "sexeLabel": "sexe",
                            "pere": "id_pere",
                            "mere": "id_mere"},
                  inplace=True)

In [14]:
print(df_personne.head())

                                       id                      nom_fr  \
0  http://www.wikidata.org/entity/Q727886  Marcus Valerius Maximianus   
1  http://www.wikidata.org/entity/Q728245   Saint Marcian of Syracuse   
2  http://www.wikidata.org/entity/Q728335      Lucius Caninius Gallus   
3  http://www.wikidata.org/entity/Q728349                   Severinus   
4  http://www.wikidata.org/entity/Q728527         Gaius Vibius Marsus   

  praenomen     nomen cognomen     gens      sexe         date_naissance  \
0    Marcus  Valerius      NaN  Valerii  masculin   0200-01-01T00:00:00Z   
1       NaN       NaN      NaN      NaN  masculin   0050-01-01T00:00:00Z   
2    Lucius  Caninius      NaN  Caninii  masculin  -0100-01-01T00:00:00Z   
3       NaN   Flavius      NaN   Flavii  masculin   0450-01-01T00:00:00Z   
4     Caius    Vibius   Marsus    Vibii  masculin  -0100-01-01T00:00:00Z   

                           lieu_naissance              date_mort  \
0   http://www.wikidata.org/entity/Q

In [15]:
df_activite = pd.read_table("query_activite.csv", delimiter =",") 
print(df_activite.head()) 

                                    item         occupationLabel
0  http://www.wikidata.org/entity/Q46762               militaire
1  http://www.wikidata.org/entity/Q46762  personnalité politique
2  http://www.wikidata.org/entity/Q46768  personnalité politique
3  http://www.wikidata.org/entity/Q46780  personnalité politique
4  http://www.wikidata.org/entity/Q46814  personnalité politique


In [16]:
df_activite.rename(columns={"item": "personne_id",
                            "occupationLabel": "activite"},
                  inplace=True)

In [17]:
df_poste = pd.read_table("query_poste.csv", delimiter =",") 
print(df_poste.head()) 

                                   item     positionLabel
0  http://www.wikidata.org/entity/Q1048            consul
1  http://www.wikidata.org/entity/Q1048           préteur
2  http://www.wikidata.org/entity/Q1048          questeur
3  http://www.wikidata.org/entity/Q1048  pontifex maximus
4  http://www.wikidata.org/entity/Q1048         dictateur


In [18]:
df_poste.rename(columns={"item": "personne_id",
                            "positionLabel": "poste"},
                  inplace=True)

In [19]:
df_oeuvre = pd.read_table("query_oeuvre.csv", delimiter =",") 
print(df_oeuvre.head()) 

                                      oeuvre  \
0   http://www.wikidata.org/entity/Q87213120   
1     http://www.wikidata.org/entity/Q660519   
2     http://www.wikidata.org/entity/Q932676   
3   http://www.wikidata.org/entity/Q19074050   
4  http://www.wikidata.org/entity/Q104789146   

                                         oeuvreLabel                  titre  \
0  Abrégé des Histoires philippiques de Trogue Po...                    NaN   
1                                     Noctes Atticae         Noctes Atticae   
2                                        Diatessaron                    NaN   
3                              Address to the Greeks  Ἐπιστολὴ πρὸς Ἕλληνας   
4                                       Controverses                    NaN   

                                   auteur    genreLabel  
0  http://www.wikidata.org/entity/Q210853           NaN  
1  http://www.wikidata.org/entity/Q294800      Q1009974  
2  http://www.wikidata.org/entity/Q272087           NaN  
3  h

In [20]:
df_oeuvre.rename(columns={"oeuvre": "id",
                          "oeuvreLabel": "titre_fr",
                          "titre": "titre_lat",
                          "auteur": "auteur_id",
                          "genreLabel": "genre"},
                  inplace=True)

In [ ]:
df_oeuvre.loc[df_oeuvre['oeuvre'].str.contains('http://www.wikidata.org/entity/Q'), 'oeuvre'] = s.replace('http://www.wikidata.org/entity/Q', '')

In [ ]:
for index, row in df_oeuvre.iterrows():
    e = df_oeuvre.at[index, 'id']
    e.replace('http://www.wikidata.org/entity/Q', '')

In [ ]:
print(df_oeuvre.head()) 

In [ ]:
print(type(df_oeuvre.at[0, 'id']))

In [ ]:
print(df_oeuvre.at[0, 'id'])

In [ ]:
e = df_oeuvre.at[0, 'id']
e.replace('http://www.wikidata.org/entity/Q', '')

In [ ]:
for index, row in df_oeuvre.iterrows():
    e = df_oeuvre.at[index, 'id']
    e.replace('http://www.wikidata.org/entity/Q', '')

In [ ]:
df_oeuvre = df_oeuvre.id.str.replace('http://www.wikidata.org/entity/Q', '', regex=True)

In [21]:
df_mariage = pd.read_table("query_mariage.csv", delimiter =",") 
print(df_mariage.head()) 

                                     item  \
0  http://www.wikidata.org/entity/Q170026   
1  http://www.wikidata.org/entity/Q170512   
2  http://www.wikidata.org/entity/Q171023   
3  http://www.wikidata.org/entity/Q171083   
4  http://www.wikidata.org/entity/Q172168   

                                    epouse  
0   http://www.wikidata.org/entity/Q232329  
1                               t291183301  
2   http://www.wikidata.org/entity/Q383304  
3  http://www.wikidata.org/entity/Q4066696  
4   http://www.wikidata.org/entity/Q234562  


In [22]:
df_mariage.insert(0, 'id', range(0, len(df_mariage)))


In [23]:
df_mariage.rename(columns={"item": "id_mari",
                          "epouse": "id_epouse"},
                  inplace=True)

In [24]:
print(df_mariage.head()) 

   id                                 id_mari  \
0   0  http://www.wikidata.org/entity/Q170026   
1   1  http://www.wikidata.org/entity/Q170512   
2   2  http://www.wikidata.org/entity/Q171023   
3   3  http://www.wikidata.org/entity/Q171083   
4   4  http://www.wikidata.org/entity/Q172168   

                                 id_epouse  
0   http://www.wikidata.org/entity/Q232329  
1                               t291183301  
2   http://www.wikidata.org/entity/Q383304  
3  http://www.wikidata.org/entity/Q4066696  
4   http://www.wikidata.org/entity/Q234562  


#### Insertion

In [25]:
sql_insert_personne = """INSERT INTO personne (id, nom_fr, praenomen, nomen, cognomen, gens, sexe, date_naissance, lieu_naissance, date_mort, lieu_mort, id_pere, id_mere) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [26]:
sql_insert_oeuvre = """INSERT INTO oeuvre (id, titre_fr, titre_lat, auteur_id, genre) VALUES (%s, %s, %s, %s, %s)"""

In [27]:
sql_insert_poste = """INSERT INTO poste (personne_id, poste) VALUES (%s, %s)"""

In [28]:
sql_insert_activite = """INSERT INTO activite (personne_id, activite) VALUES (%s, %s)"""

In [29]:
sql_insert_mariage = """INSERT INTO mariage (id, id_mari, id_epouse) VALUES (%s, %s, %s)"""

In [30]:
conn = psycopg2.connect(
   database="projetspqr", user='postgres', password=mot_passe, host='localhost', port= '5432'
)
conn.autocommit = True

cursor = conn.cursor()

for index, row in df_oeuvre.iterrows():
    cursor.execute(sql_insert_oeuvre, tuple(row))

In [31]:
for index, row in df_personne.iterrows():
    cursor.execute(sql_insert_personne, tuple(row))

In [32]:
for index, row in df_activite.iterrows():
    cursor.execute(sql_insert_activite, tuple(row))


In [33]:
for index, row in df_poste.iterrows():
    cursor.execute(sql_insert_poste, tuple(row))


In [34]:
for index, row in df_mariage.iterrows():
    cursor.execute(sql_insert_mariage, tuple(row))